# Deploy a PyTorch model to a Clipper cluster for use by other applications


## Load libraries

In [14]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

In [29]:
wine_data = pd.read_csv('data/wine_data.csv')

#### Collecting Features

In [30]:
wine_features = wine_data.drop('Class', axis = 1)
wine_target = wine_data[['Class']]

In [31]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(wine_features,
                                                    wine_target,
                                                    test_size=0.4,
                                                    random_state=0)

In [32]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(x_test.values).float()
Ytrain_ = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest_ = torch.from_numpy(y_test.values).view(1,-1)[0]

## Creating a classifier


In [33]:
input_size = 13
output_size = 3
hidden_size = 100

In [34]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [35]:
model = Net()

In [36]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.NLLLoss()

#### Training the model

In [37]:
epochs = 1000

for epoch in range(epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    if epoch % 100 == 0:
        print ('Epoch', epoch, 'loss', loss.item())

Epoch 0 loss 1.1848348379135132
Epoch 100 loss 0.2554129362106323
Epoch 200 loss 0.043856181204319
Epoch 300 loss 0.2409471869468689
Epoch 400 loss 0.031679265201091766
Epoch 500 loss 0.025489557534456253
Epoch 600 loss 0.021492188796401024
Epoch 700 loss 0.018933657556772232
Epoch 800 loss 0.01718682236969471
Epoch 900 loss 0.015777599066495895


## Install Clipper Admin

In [193]:
!pip install clipper_admin

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [194]:
# Clipper pytorch container has version 0.4.0. Only supports models trained and saved in pytorch versions <0.4.0 (Latest pytorch version is 1.0.1) 
# Need to install pytorch 0.4.0, re-run first demo and get a model in 0.4.0 before continuing.

 ## Create a Clipper Connection 
* create a ClipperConnection object  with the type of ContainerManager you want to use. In this case, you will be using the DockerContainerManager.

In [195]:
!docker -v

Docker version 18.06.0-ce, build 0ffa825


In [20]:
from clipper_admin import ClipperConnection, DockerContainerManager
clipper_conn = ClipperConnection(DockerContainerManager())

## Start a Clipper cluster.

* The following command will start 3 Docker containers:

* The Query Frontend: The Query Frontend container listens for incoming prediction requests and schedules and routes them to the deployed models.
* The Management Frontend: The Management Frontend container manages and updates the cluster’s internal configuration state, such as tracking which models are deployed and which application endpoints have been registered.
* A Redis instance: Redis is used to persistently store Clipper’s internal configuration state. By default, Redis is started on port 6380 instead of the standard Redis default port 6379 to avoid collisions with any Redis instances that are already running.

In [21]:
clipper_conn.start_clipper()
clipper_addr = clipper_conn.get_query_addr()

19-06-06:12:41:29 INFO     [docker_container_manager.py:119] Starting managed Redis instance in Docker
19-06-06:12:41:33 INFO     [clipper_admin.py:126] Clipper is running


In [22]:
!docker ps --filter label=ai.clipper.container.label

CONTAINER ID        IMAGE                               COMMAND                  CREATED             STATUS                  PORTS                                            NAMES
a11897a412a1        prom/prometheus:v2.1.0              "/bin/prometheus --c…"   2 seconds ago       Up Less than a second   0.0.0.0:9090->9090/tcp                           metric_frontend-42496
7e132a7e3a9b        clipper/frontend-exporter:0.3.0     "python /usr/src/app…"   2 seconds ago       Up 1 second                                                              query_frontend_exporter-88145
fe66686ca08e        clipper/query_frontend:0.3.0        "/clipper/release/sr…"   3 seconds ago       Up 2 seconds            0.0.0.0:1337->1337/tcp, 0.0.0.0:7000->7000/tcp   query_frontend-88145
2d24f3265976        clipper/management_frontend:0.3.0   "/clipper/release/sr…"   4 seconds ago       Up 2 seconds            0.0.0.0:1338->1338/tcp                           mgmt_frontend-52500
fc650c5f7968        redis:alpin

## Create an Application
*  slo_micros : The query latency objective for the application in microseconds. This is the processing latency between Clipper receiving a request and sending a response. 
*  default_output : The default output for the application. The default output will be returned whenever an application is unable to receive a response from a model within the specified query latency SLO

In [23]:
app_name = "wine-classsifier-model"
default_output = "default"

In [24]:
clipper_conn.register_application(
    name = app_name,
    input_type = "floats",
    default_output = default_output,
    slo_micros=10000000)

19-06-06:12:41:34 INFO     [clipper_admin.py:201] Application wine-classsifier-model was successfully registered


In [25]:
clipper_conn.get_all_apps()


['wine-classsifier-model']

In [38]:
model.eval()

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

## Define a method for prediction
* This is required as an argument for the pytorch model deployer in clipper

In [39]:
def predict_torch_model(model, data):


    sample_tensor = torch.from_numpy(data).float()


    out = model(sample_tensor)

    _, predicted = torch.max(out.data, -1)

        
    return ["The wine belongs to class - " + str(predicted.item())]

## Deploy model to Clipper
* Clipper has a pytorch specific deployer
* Clipper must download this Docker image from the internet, so this may take a minute
* Link the generated pytorch-model to the application we created before.
* Batch_size is 1 as we are only sending in one line of input data for prediction with one request

In [40]:
from clipper_admin.deployers import pytorch as pytorch_deployer

pytorch_deployer.deploy_pytorch_model(
    clipper_conn,
    name = "pytorch-model",
    version = 1, 
    input_type = "floats", 
    func = predict_torch_model, # predict function wrapper
    pytorch_model = model, # pass model to function
    batch_size = 1 
)

19-06-06:12:42:10 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmpcc1ayzos
19-06-06:12:42:10 INFO     [deployer_utils.py:54] Serialized and supplied predict function
19-06-06:12:42:10 INFO     [pytorch.py:204] Torch model saved
19-06-06:12:42:10 INFO     [pytorch.py:217] Using Python 3.6 base image
19-06-06:12:42:10 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmpcc1ayzos
19-06-06:12:42:11 INFO     [clipper_admin.py:456] {'stream': 'Step 1/2 : FROM clipper/pytorch36-container:0.3.0'}
19-06-06:12:42:11 INFO     [clipper_admin.py:456] {'stream': '\n'}
19-06-06:12:42:11 INFO     [clipper_admin.py:456] {'stream': ' ---> 37545e712105\n'}
19-06-06:12:42:11 INFO     [clipper_admin.py:456] {'stream': 'Step 2/2 : COPY /tmp/clipper/tmpcc1ayzos /model/'}
19-06-06:12:42:11 INFO     [clipper_admin.py:456] {'stream': '\n'}
19-06-06:12:42:11 INFO     [clipper_admin.py:456] {'stream': ' ---> dbc242cb44e0\n'}
19-06-06:12:42:11 INFO    

In [41]:
clipper_conn.link_model_to_app(app_name="wine-classsifier-model", model_name="pytorch-model")

19-06-06:12:42:21 INFO     [clipper_admin.py:263] Model pytorch-model is now linked to application wine-classsifier-model


## Query the application
* Query frontend is running at 1337 as we saw from the ps command before

In [44]:
import requests
import json


clipper_addr = 'localhost:1337'

data = [12.82,3.37,2.3,19.5,88,1.48,0.66,0.4,0.97,10.26,0.72,1.75,685]


req_json = json.dumps({ "input": [np.float64(x) for x in data]})

response = requests.post(
           "http://%s/%s/predict" % (clipper_addr, 'wine-classsifier-model'),
           headers={"Content-type": "application/json"},
           data=req_json)
  
print(response.json())

{'query_id': 2, 'output': 'The wine belongs to class - 2', 'default': False}


## Stop Clipper

In [45]:
clipper_conn.stop_all()

19-06-06:12:43:39 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers
